In [34]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler


In [35]:
stocks = {}
for sector in os.listdir():
    if "." in sector : continue
    stocks[sector] = {}
    for file in os.listdir(sector + "/"):
        if file == ".DS_Store" or file =="news.ipynb": continue
        stock = file[:file.index(".")]
        stocks[sector][stock] = pd.read_csv(sector + "/" + file)
        
        

In [36]:
stocks

{'Basic_Materials': {'AMWD':             Date Close/Last  Volume     Open      High       Low
  0     11/29/2024     $90.78   92478   $90.59    $91.40    $90.35
  1     11/27/2024     $89.19  224561   $92.00    $94.29    $88.78
  2     11/26/2024     $91.62  545068   $94.49    $97.10    $90.79
  3     11/25/2024    $100.86  219650   $98.37   $103.50    $98.37
  4     11/22/2024     $97.24  173068   $95.68  $98.0313    $95.25
  ...          ...        ...     ...      ...       ...       ...
  1253  12/06/2019    $104.52  130922  $104.00   $105.38  $103.915
  1254  12/05/2019    $103.26  179655  $103.38   $103.91   $102.50
  1255  12/04/2019    $102.98  121946  $103.22   $104.31   $102.74
  1256  12/03/2019    $102.69  143640  $101.16   $103.54   $101.05
  1257  12/02/2019    $102.35  166752  $102.84   $102.91   $101.20
  
  [1258 rows x 6 columns],
  'ATCOL':            Date Close/Last   Volume      Open      High       Low
  0    11/27/2024     $25.08   4559.0    $25.04    $25.10    $

In [38]:
test_stock = stocks['Industrials']['FER']

In [39]:
test_stock

,Date,Close/Last,Volume,Open,High,Low
0,11/29/2024,$41.37,35890.0,$40.95,$41.40,$40.95
1,11/27/2024,$41.12,24765.0,$41.18,$41.30,$41.01
2,11/26/2024,$40.71,31938.0,$40.48,$40.71,$40.29
3,11/25/2024,$40.45,41856.0,$40.82,$40.87,$40.245
4,11/22/2024,$38.76,45444.0,$38.79,$39.00,$38.71
...,...,...,...,...,...,...
337,07/31/2023,$33.40,NaN,$33.40,$33.40,$33.40
338,07/28/2023,$33.40,NaN,$33.40,$33.40,$33.40
339,07/27/2023,$33.40,NaN,$33.40,$33.40,$33.40
340,07/26/2023,$33.40,NaN,$33.40,$33.40,$33.40


In [40]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

df = test_stock
num_data = df[['Open','Close/Last','High','Low']].to_numpy().astype(str)
num_data = np.char.replace(num_data, '$', '')
num_data = num_data.astype(float)
num_data = num_data.transpose()

# Define the time window for momentum calculation
window = 15  # Number of days for momentum calculation
prices = num_data[1][-window:]  # Select the last 'window' days of prices

# Prepare X (time) and y (prices)
X = np.arange(len(prices)).reshape(-1, 1)  # Time points as independent variable
y = prices  # Closing prices as dependent variable

# Apply Linear Regression
model = LinearRegression()
model.fit(X, y)

# Extract the slope (price momentum)
momentum = model.coef_[0]
print(prices)
print(f"Price Momentum (Slope): {momentum:.2f}")


[33.1 33.1 33.1 33.1 33.1 33.1 33.1 33.4 33.4 33.4 33.4 33.4 33.4 33.4
 33.4]
Price Momentum (Slope): 0.03


### Calculating features

In [9]:
import numpy as np

post_data = []
for sector in stocks:
    stock_list = stocks[sector]
    # print(sector)
    stock_feature = []
    for stock in stock_list:
        # print(stock)
        df = stock_list[stock]
        num_data = df[['Open','Close/Last','High','Low']].to_numpy().astype(str)
        num_data = np.char.replace(num_data, '$', '')
        num_data = num_data.astype(float)
        num_data = num_data.transpose()
        #Avg Daily volume

        avg_volume = df['Volume'].mean()

        #Price Volatility
        year_data = num_data[-252:]
        shift_close = np.roll(year_data, 1)  # Shifts all elements by one position
        shift_close = year_data[0]
        daily_ret = np.log(year_data/shift_close)

        std_dev = np.std(daily_ret)
        volatility = std_dev * np.sqrt(252)

        #Average Daily Return

        change = num_data[1] - num_data[0]

        pc_change = (change/num_data[0]) * 100
        pc_change = pc_change.round(3)
        

        # Trend Consistency (Price Direction)

        trend = (pc_change > 0).astype(int)
        pos,neg = trend.sum(),len(trend) - trend.sum()
        if neg == 0: neg = 0.5
        trend_cons = pos/neg

        #Avg Close

        avg_close = num_data[1].mean()
        avg_open = num_data[0].mean()
        avg_high = num_data[2].mean()
        avg_low = num_data[3].mean()

        # Price Momentum

        momentums = []
        for window in [30,90,180,365]:
            prices = num_data[1][-window:]  # Select the last 'window' days of prices
            X,y = np.arange(len(prices)).reshape(-1, 1), prices  

            # Apply Linear Regression
            model = LinearRegression()
            model.fit(X, y)

            # Extract the slope (price momentum)
            momentums.append(model.coef_[0])
        # print(momentums)
        # Combining the data

        stock_feature.append([
            stock,
            sector,
            avg_volume,
            volatility,
            pc_change.mean(),
            trend_cons,
            avg_close,
            avg_open,
            avg_high,
            avg_low
        ] + momentums)
    post_data.extend(stock_feature)


stock_features = pd.DataFrame(post_data,columns=['stock','sector','avg_volume','volatility','pc_change','trend_consistency','avg_close','avg_open','avg_high','avg_low',"momentum_1m","momentum_3m","momentum_6m","momentum_1y"])
onehot = pd.get_dummies(stock_features['sector'], prefix='sector').astype(int)
stock_features = pd.concat([stock_features.drop(columns=['sector']),onehot],axis=1)

scaler = MinMaxScaler()
minmax_features = ['avg_volume','volatility','pc_change','trend_consistency'] + stock_features.columns.to_list()[9:]
stock_features[minmax_features] = scaler.fit_transform(stock_features[minmax_features])

logminxmax_features = stock_features.columns.to_list()[5:9]
stock_features[logminxmax_features] = np.log(stock_features[logminxmax_features])
stock_features[logminxmax_features] = scaler.fit_transform(stock_features[logminxmax_features])

In [10]:
stock_features

,stock,avg_volume,volatility,pc_change,trend_consistency,avg_close,avg_open,avg_high,avg_low,momentum_1m,...,sector_Consumer_Discretionary,sector_Consumer_Staples,sector_Energy,sector_Finance,sector_Healthcare,sector_Industrials,sector_Real_Estate,sector_Technology,sector_Telecommunications,sector_Utilities
0,AMWD,0.000275,0.433489,0.745353,0.784876,0.492789,0.492571,0.492909,0.492658,0.666413,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ATCOL,0.000000,0.000000,0.782510,0.318904,0.333959,0.333537,0.331709,0.336008,0.675232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CRML,0.000084,1.000000,1.000000,0.067306,0.203798,0.203115,0.203121,0.204098,0.676994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,GSM,0.003104,0.903248,0.764233,0.614817,0.083829,0.083324,0.085029,0.082480,0.676511,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IPX,0.000097,0.784644,0.526109,0.419015,0.207603,0.207253,0.208684,0.206552,0.682067,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,LNT,0.003282,0.161664,0.787738,0.791864,0.447085,0.446752,0.445941,0.448023,0.670054,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
103,NFE,0.002847,0.636973,0.666590,0.635728,0.368104,0.367940,0.369294,0.366932,0.679379,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
104,NWE,0.000802,0.197424,0.739345,0.691573,0.453716,0.453458,0.452733,0.454526,0.677188,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
105,OTTR,0.000392,0.253660,0.760228,0.727251,0.469107,0.468842,0.468385,0.469680,0.669052,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
for sector in post_data:
    post_data[sector].to_csv("stock_data/"+sector+".csv")

TypeError: list indices must be integers or slices, not list

In [12]:
stock_features.to_csv("stock_features.csv")

### Finalized Function

In [ ]:
def get_stock_data(dir):
    """
    dir (str) : directory of folder containing sectors subfolders containing stock data files.
    
    return 
    """
    stocks = {}
    for sector in os.listdir(dir):
        if "." in sector : continue
        stocks[sector] = {}
        for file in os.listdir(dir+"/"+sector + "/"):
            if file == ".DS_Store" or file =="news.ipynb": continue
            stock = file[:file.index(".")]
            stocks[sector][stock] = pd.read_csv(dir+"/"+sector + "/" + file)
    
    return stocks

def get_stock_feature(stocks,month,year,timeframe = 90):
    """
    stocks (dict)   : stocks dict. keys is sector , each sector contain another dict with keys consisting of stock tickers and value being price df.
    timeframe (int) : number of days to consider duuring rolling windows [1,15,30,90,180,365] 
    """
    post_data = []
    for sector in stocks:
        stock_list = stocks[sector]
        # print(sector)
        stock_feature = []
        for stock in stock_list:
            # print(stock)
            df = stock_list[stock]
            df['Date'] = pd.to_datetime(df['Date'])
            filtered_dates = df[(df['Date'].dt.month == month) & (df['Date'].dt.year == year)]
            first_date = filtered_dates['Date'].min()

            last_month = month + (timeframe/30)
            last_year = year
            if last_month > 12:
                month -= 12
                last_year += 1
            filtered_dates = df[(df['Date'].dt.month == last_month) & (df['Date'].dt.year == last_year)]
            last_date = filtered_dates['Date'].min()

            df = df[(df['Date'] >= first_date)]

            # if df is empty, there is no stock data at the begin date
            if df.empty: continue

            # if there is not enough data to fit the timeframe, an error is raised
            if df.shape[0] < timeframe:
                raise Exception(f"Timeframe is too large. Timeframe is of size {timeframe} while df only contains {df.shape[0]} rows")
            
            num_data = df[['Open','Close/Last','High','Low']].to_numpy().astype(str)
            num_data = np.char.replace(num_data, '$', '')
            num_data = num_data.astype(float)
            num_data = num_data.transpose()
            
            #Avg Daily volume
            avg_volume = df.head(timeframe)['Volume'].mean()


            #Average Daily Return
            change = num_data[1][:timeframe] - num_data[0][:timeframe]
            pc_change = (change/num_data[0][:timeframe]) * 100
            pc_change = pc_change.round(3)

            # Trend Consistency (Price Direction)
            trend = (pc_change > 0).astype(int)
            pos,neg = trend.sum(),len(trend) - trend.sum()
            if neg == 0: neg = 0.5
            trend_cons = pos/neg

            #Avg Close
            avg_close = num_data[1][:timeframe].mean()
            avg_open = num_data[0][:timeframe].mean()
            avg_high = num_data[2][:timeframe].mean()
            avg_low = num_data[3][:timeframe].mean()

            #Price Volatility
            sum_top = num_data[1][:timeframe] - avg_close
            volatility = np.sqrt(np.power(sum_top,2).sum() / timeframe)

            # Price Momentum
            momentums = []
            for window in [30,90,180,365]:
                if len(num_data[1]) < window:
                    momentums.append(np.nan)
                    break
                prices = num_data[1][-window:]  # Select the last 'window' days of prices
                X,y = np.arange(len(prices)).reshape(-1, 1), prices  
                # Apply Linear Regression
                model = LinearRegression()
                model.fit(X, y)
                # Extract the slope (price momentum)
                momentums.append(model.coef_[0])

            # Combining the data

            stock_feature.append([
                stock,
                sector,
                avg_volume,
                volatility,
                pc_change.mean(),
                trend_cons,
                avg_close,
                avg_open,
                avg_high,
                avg_low
            ] + momentums)
        post_data.extend(stock_feature)

    # column names for momentums are hardcoded

    stock_features = pd.DataFrame(post_data,columns=['stock','sector','avg_volume','volatility','pc_change','trend_consistency','avg_close','avg_open','avg_high','avg_low',"momentum_1m","momentum_3m","momentum_6m","momentum_1y"])
    onehot = pd.get_dummies(stock_features['sector'], prefix='sector').astype(int)
    stock_features = pd.concat([stock_features.drop(columns=['sector']),onehot],axis=1)
    return stock_features

def normalize_features(stock_features):

    scaler = MinMaxScaler()

    #MinMax for sector, avg_volume, pc_change, trend_consistency, momentums
    minmax_features = ['avg_volume','pc_change','trend_consistency'] + stock_features.columns.to_list()[9:]
    stock_features[minmax_features] = scaler.fit_transform(stock_features[minmax_features])
    
    #LogMinMax for avg_close,open,high,low,volatility
    logminxmax_features = stock_features.columns.to_list()[5:9] + ['volatility']
    stock_features[logminxmax_features] = np.log(stock_features[logminxmax_features])
    stock_features[logminxmax_features] = scaler.fit_transform(stock_features[logminxmax_features])

    return stock_features

In [49]:
df = stock_list[stock]

# df['Date'] = pd.to_datetime(df['Date'])

target_month = 1  # January
target_year = 2023

filtered_dates = df[(df['Date'].dt.month == target_month) & (df['Date'].dt.year == target_year)]

# Find the first date
first_date = filtered_dates['Date'].min()

In [50]:
first_date

Timestamp('2023-01-03 00:00:00')

In [82]:
sf = get_stock_feature(stocks,1,2023,90)

normalize_features(sf)

,stock,avg_volume,volatility,pc_change,trend_consistency,avg_close,avg_open,avg_high,avg_low,momentum_1m,...,sector_Consumer_Discretionary,sector_Consumer_Staples,sector_Energy,sector_Finance,sector_Healthcare,sector_Industrials,sector_Real_Estate,sector_Technology,sector_Telecommunications,sector_Utilities
0,AMWD,0.000641,0.431212,0.569048,0.345930,0.511469,0.511252,0.510719,0.512284,0.886929,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ATCOL,0.000000,0.000000,0.627139,0.014831,0.342965,0.342372,0.339383,0.345854,0.910948,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CRML,0.000038,0.279365,0.046495,0.000000,0.188907,0.189212,0.188914,0.188975,0.908845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,GSM,0.004384,0.099520,0.329910,0.062500,0.121128,0.120821,0.119317,0.122726,0.908242,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IPX,0.000110,0.421798,1.000000,0.242021,0.315583,0.314470,0.314321,0.315143,0.904949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,LNT,0.004966,0.337428,0.665260,0.291667,0.453146,0.452653,0.450719,0.454970,0.915366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
101,NFE,0.017240,0.380530,0.233046,0.242021,0.240114,0.240406,0.240769,0.239543,0.901000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
102,NWE,0.001254,0.294006,0.535506,0.345930,0.442815,0.442538,0.440471,0.444754,0.915902,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
103,OTTR,0.000820,0.461021,0.360551,0.154412,0.495471,0.495621,0.494331,0.496809,0.897041,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### get random stock

In [89]:
sf = sf.set_index("stock")

In [129]:
import random

rand_stock = pd.DataFrame([sf.iloc[random.randint(0,len(sf.loc['LNT']))] for i in range(3)])

rand_stock

,avg_volume,volatility,pc_change,trend_consistency,avg_close,avg_open,avg_high,avg_low,momentum_1m,momentum_3m,...,sector_Consumer_Discretionary,sector_Consumer_Staples,sector_Energy,sector_Finance,sector_Healthcare,sector_Industrials,sector_Real_Estate,sector_Technology,sector_Telecommunications,sector_Utilities
COKE,0.000148,0.733793,0.729410,0.196429,0.845836,0.845834,0.845606,0.846050,0.817942,0.663635,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CTAS,0.004655,0.530931,0.628006,0.583333,0.612422,0.612229,0.610690,0.613838,0.910079,0.869423,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IPX,0.000110,0.421798,1.000000,0.242021,0.315583,0.314470,0.314321,0.315143,0.904949,0.878770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cosine sim

In [121]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(sf, rand_stock)
similarities = cos_sim.transpose()

top_matches = []
for idx, sim in enumerate(similarities):

    top_3_idx = np.argsort(sim)[-4:][::-1]  
    top_3_idx = top_3_idx[1:]
    for rank, match_idx in enumerate(top_3_idx):
        top_matches.append({
            'UserStock': rand_stock[idx].name,
            'MatchedStock': sf.iloc[match_idx].name,
            'Similarity': sim[match_idx],
            'Rank': rank + 1
        })

# Convert to DataFrame for better readability
cos_top_matches_df = pd.DataFrame(top_matches)

In [122]:
cos_top_matches_df

,UserStock,MatchedStock,Similarity,Rank
0,IPX,SGML,0.985627,1
1,IPX,AMWD,0.970064,2
2,IPX,ATCOL,0.968696,3
3,AMZN,PDD,0.995940,1
4,AMZN,SBUX,0.994186,2
5,AMZN,CTAS,0.990084,3
6,COKE,PEP,0.959268,1
7,COKE,CCEP,0.934626,2
8,COKE,MDLZ,0.933139,3


### Euclidean

In [136]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import pairwise_distances

distances = pairwise_distances(rand_stock, sf, metric='euclidean')

# Create a DataFrame for distances
distances_df = pd.DataFrame(distances, index=rand_stock.index, columns=sf.index).drop(columns=rand_stock.index.to_list())


In [137]:
closest_stocks = distances_df.idxmin(axis=1)
closest_stocks

COKE     PEP
CTAS     MAR
IPX     SGML
dtype: object

#### Using KNN

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='euclidean')
model.fit(sf)

distances, indices = model.kneighbors(rand_stock, n_neighbors=3)

print(indices.flatten())
closest_stocks = [sf.iloc[i].name for i in indices.flatten()]
for s in rand_stock.index:
    print(s)
    if s in closest_stocks:
        closest_stocks.remove(s)
print("Closest stocks:", closest_stocks)

[22 27 20 13 14 17  4  7  0]
COKE
CTAS
IPX
Closest stocks: ['PEP', 'CCEP', 'MAR', 'PDD', 'SGML', 'AMWD']
